In [34]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
from itertools import chain, repeat
from pprint import pprint
from typing import (
    AbstractSet, Any, Callable, Collection, Dict, Generator, Generic, Iterable, Mapping,
    NamedTuple, Optional, Tuple, TypeVar, Union
)

from graphql import GraphQLList, GraphQLString, parse
from graphql.utils.build_ast_schema import build_ast_schema
from graphql_compiler.compiler.compiler_frontend import graphql_to_ir
from graphql_compiler.compiler.blocks import (
    Backtrack, CoerceType, ConstructResult, Filter, GlobalOperationsStart, MarkLocation, 
    QueryRoot, Traverse
)
from graphql_compiler.compiler.compiler_entities import BasicBlock, Expression
from graphql_compiler.compiler.expressions import (
    BinaryComposition, ContextField, Literal, LocalField, OutputContextField, Variable
)
from graphql_compiler.compiler.compiler_frontend import IrAndMetadata, graphql_to_ir
from graphql_compiler.compiler.helpers import Location, get_only_element_from_collection
from graphql_compiler.schema import GraphQLDate, GraphQLDateTime, GraphQLDecimal
from graphql_compiler.tests.test_helpers import SCHEMA_TEXT

In [24]:
FilterInfo = namedtuple(
    'FilterInfo',
    ('field_name', 'op_name', 'value'),
)
DataToken = TypeVar('DataToken')


class LineageToken(NamedTuple):
    token: DataToken
    lineage_by_location: Dict[Location, DataToken]

        
def make_empty_lineage_token(token: DataToken):
    return LineageToken(token=token, lineage_by_location=dict())
        

class InterpreterAdapter(Generic[DataToken], metaclass=ABCMeta):
    @abstractmethod
    def get_tokens_of_type(
        self,
        type_name: str, 
        **hints
    ) -> Iterable[DataToken]:
        pass

    @abstractmethod
    def project_property(
        self,
        lineages_and_other_data: Iterable[Tuple[LineageToken, Any]], 
        field_name: str,
        **hints
    ) -> Iterable[Tuple[LineageToken, Any]]:
        pass

    @abstractmethod
    def project_neighbors(
        self,
        lineages_and_other_data: Iterable[Tuple[LineageToken, Any]], 
        direction: str,
        edge_name: str, 
        **hints
    ) -> Iterable[Tuple[LineageToken, Iterable[DataToken], Any]]:
        # If using a generator instead of a list for the Iterable[DataToken] part,
        # be careful -- generators are not closures! Make sure any state you pull into
        # the generator from the outside does not change, or that bug will be hard to find.
        # Remember: it's always safer to use a function to produce the generator, since
        # that will explicitly preserve all the external values passed into it.
        pass

    @abstractmethod
    def coerce_to_type(
        self,
        lineages_and_other_data: Iterable[Tuple[LineageToken, Any]], 
        type_name: str,
        **hints
    ) -> Iterable[Tuple[LineageToken, Any]]:
        pass

In [21]:
def _apply_operator(operator: str, left_value: Any, right_value: Any) -> Any:
    if operator == '=':
        return left_value == right_value
    elif operator == 'contains':
        return right_value in left_value
    else:
        raise NotImplementedError()

In [3]:
def _evaluate_binary_composition(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    expression: Expression,
    lineages_and_other_data: Iterable[Tuple[LineageToken, Any]],
) -> Iterable[Tuple[LineageToken, Any]]:
    lineages_and_other_data = _evaluate_expression(adapter, query_arguments, expression.left, lineages_and_other_data)
    lineages_and_other_data = _evaluate_expression(adapter, query_arguments, expression.right, lineages_and_other_data)

    return (
        (lineage, (_apply_operator(expression.operator, left_value, right_value), other_data))
        for lineage, (right_value, (left_value, other_data)) in lineages_and_other_data
    )

        
def _evaluate_local_field(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: LocalField,
    lineages_and_other_data: Iterable[Tuple[LineageToken, Any]],
) -> Iterable[Tuple[LineageToken, Any]]:
    field_name = expression.field_name
    return adapter.project_property(lineages_and_other_data, field_name)


def _evaluate_context_field(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: Union[ContextField, OutputContextField],
    lineages_and_other_data: Iterable[Tuple[LineageToken, Any]],
) -> Iterable[Tuple[LineageToken, Any]]:
    location = expression.location.at_vertex()
    field_name = expression.location.field
    
    rotated_lineages_and_other_data = (
        (
            LineageToken(
                token=lineage.lineage_by_location[location],
                lineage_by_location=lineage.lineage_by_location
            ),
            (lineage, other_data)
        )
        for lineage, other_data in lineages_and_other_data
    )
    
    return (
        (lineage, (value, other_data))
        for _, (value, (lineage, other_data)) in adapter.project_property(
            rotated_lineages_and_other_data, field_name
        )
    )

    
def _evaluate_variable(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    expression: Variable,
    lineages_and_other_data: Iterable[Tuple[LineageToken, Any]],
) -> Iterable[Any]:
    variable_value = query_arguments[expression.variable_name[1:]]
    return (
        (lineage, (variable_value, other_data))
        for lineage, other_data in lineages_and_other_data
    )


def _evaluate_expression(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    expression: Expression,
    lineages_and_other_data: Iterable[Tuple[LineageToken, Any]],
) -> Iterable[Tuple[LineageToken, Any]]:
    type_to_handler = {
        BinaryComposition: _evaluate_binary_composition,
        ContextField: _evaluate_context_field,
        OutputContextField: _evaluate_context_field,
        LocalField: _evaluate_local_field,
        Variable: _evaluate_variable,
    }
    expression_type = type(expression)
    return type_to_handler[expression_type](adapter, query_arguments, expression, lineages_and_other_data)

In [26]:
def _handle_filter(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    block: Filter,
    lineages: Iterable[LineageToken],
) -> Iterable[LineageToken]:
    predicate = block.predicate
    
    lineages_and_data = (
        (lineage, None)
        for lineage in lineages
    )
    
    yield from (
        lineage
        for lineage, (predicate_value, _) in _evaluate_expression(
            adapter, query_arguments, predicate, lineages_and_data)
        if predicate_value
    )
    

def _handle_traverse(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: Traverse,
    lineages: Iterable[LineageToken],
) -> Iterable[LineageToken]:
    if block.optional:
        raise NotImplementedError()
    
    lineages_and_data = (
        (lineage, None)
        for lineage in lineages
    )
    
    neighbor_batches = adapter.project_neighbors(
        lineages_and_data, block.direction, block.edge_name)
    for lineage, neighbor_tokens, _ in neighbor_batches:
        yield from (
            LineageToken(token=neighbor_token, lineage_by_location=lineage.lineage_by_location)
            for neighbor_token in neighbor_tokens
        )

In [5]:
def _add_key_value_to_dict(result_dict, key, value):
    result_dict[key] = value
    return result_dict


def _produce_output(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    output_name: str,
    output_expression: Expression,
    lineages_and_data: Iterable[Tuple[LineageToken, Any]],
) -> Iterable[Tuple[LineageToken, Any]]:
    lineages_and_data = _print_tap(
        'outputting ' + output_name, lineages_and_data)
    
    return (
        (lineage, _add_key_value_to_dict(result_dict, output_name, value))
        for lineage, (value, result_dict) in _evaluate_expression(
            adapter, query_arguments, output_expression, lineages_and_data)
    )
    

def _handle_construct_result(
    adapter: InterpreterAdapter[DataToken], 
    query_arguments: Dict[str, Any],
    block: ConstructResult,
    lineages: Iterable[LineageToken],
) -> Iterable[Dict[str, Any]]:
    output_fields = block.fields
    
    lineages_and_data = (
        (lineage, dict())
        for lineage in lineages
    )
    
    for output_name, output_expression in output_fields.items():
        lineages_and_data = _produce_output(
            adapter, query_arguments, output_name, output_expression, lineages_and_data)
        
    return (
        result_dict
        for _, result_dict in lineages_and_data
    )

In [6]:
def _handle_coerce_type(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: CoerceType,
    lineages: Iterable[LineageToken],
) -> Iterable[LineageToken]:
    coercion_type = get_only_element_of_collection(block.target_class)
    return adapter.coerce_to_type(lineages, coercion_type)
    

def _handle_mark_location(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: MarkLocation,
    lineages: Iterable[LineageToken],
) -> Iterable[LineageToken]:
    current_location = block.location
    for lineage_token in lineages:
        lineage_by_location = dict(lineage_token.lineage_by_location)
        lineage_by_location[current_location] = lineage_token.token
        yield LineageToken(
            token=lineage_token.token,
            lineage_by_location=lineage_by_location,
        )
        

def _handle_backtrack(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: Backtrack,
    lineages: Iterable[LineageToken],
) -> Iterable[LineageToken]:
    backtrack_location = block.location
    for lineage_token in lineages:
        yield LineageToken(
            token=lineage_token.lineage_by_location[backtrack_location],
            lineage_by_location=lineage_token.lineage_by_location,
        )

In [7]:
def _handle_block(
    adapter: InterpreterAdapter[DataToken],
    query_arguments: Dict[str, Any],
    block: BasicBlock,
    lineages: Iterable[LineageToken],
) -> Iterable[LineageToken]:
    no_op_types = (GlobalOperationsStart,)
    if isinstance(block, no_op_types):
        return lineages
    
    lineages = _print_tap('pre: ' + str(block), lineages)
    
    handler_functions = {        
        CoerceType: _handle_coerce_type,
        Filter: _handle_filter,
        MarkLocation: _handle_mark_location,
        Traverse: _handle_traverse,
        Backtrack: _handle_backtrack,
    }
    return handler_functions[type(block)](adapter, query_arguments, block, lineages)


In [32]:
def _print_tap(info: str, lineages: Iterable[LineageToken]) -> Iterable[LineageToken]:
    return lineages
#     print('\n')
#     unique_id = hash(info)
#     print(unique_id, info)
#     from funcy.py3 import chunks
#     for lineage_chunk in chunks(100, lineages):
#         for lineage in lineage_chunk:
#             pprint((unique_id, lineage))
#             yield lineage
        

In [9]:
def interpret_ir(
    adapter: InterpreterAdapter[DataToken], 
    ir_and_metadata: IrAndMetadata, 
    query_arguments: Dict[str, Any]
) -> Iterable[Dict[str, Any]]:
    ir_blocks = ir_and_metadata.ir_blocks
    query_metadata_table = ir_and_metadata.query_metadata_table
    
    if not ir_blocks:
        raise AssertionError()
        
    first_block = ir_blocks[0]
    if not isinstance(first_block, QueryRoot):
        raise AssertionError()
        
    last_block = ir_blocks[-1]
    if not isinstance(last_block, ConstructResult):
        raise AssertionError()
        
    middle_blocks = ir_blocks[1:-1]
        
    start_class = get_only_element_from_collection(first_block.start_class)
    
    current_lineages = (
        make_empty_lineage_token(token)
        for token in adapter.get_tokens_of_type(start_class)
    )
    
    current_lineages = _print_tap('starting lineages', current_lineages)
    
    for block in middle_blocks:
        current_lineages = _handle_block(adapter, query_arguments, block, current_lineages)
        
    current_lineages = _print_tap('ending lineages', current_lineages)
        
    return _handle_construct_result(
        adapter, query_arguments, last_block, current_lineages)
    

In [25]:
vertices = {
    'Animal': [
        {'name': 'Scooby Doo', 'uuid': '1001'},
        {'name': 'Hedwig', 'uuid': '1002'},
        {'name': 'Beethoven', 'uuid': '1003'},
        {'name': 'Pongo', 'uuid': '1004'},
        {'name': 'Perdy', 'uuid': '1005'},
        {'name': 'Dipstick', 'uuid': '1006'},
        {'name': 'Dottie', 'uuid': '1007'},
        {'name': 'Domino', 'uuid': '1008'},
        {'name': 'Little Dipper', 'uuid': '1009'},
        {'name': 'Oddball', 'uuid': '1010'},
    ],
}
edges = {
    'Animal_ParentOf': [
        ('1004', '1006'),
        ('1005', '1006'),
        ('1006', '1008'),
        ('1006', '1009'),
        ('1006', '1010'),
        ('1007', '1008'),
        ('1007', '1009'),
        ('1007', '1010'),
    ],
}

vertices_by_uuid = {
    vertex['uuid']: vertex
    for vertex in chain.from_iterable(vertices.values())
}


class InMemoryAdapter(InterpreterAdapter[dict]):
    def get_tokens_of_type(
        self,
        type_name: str, 
        **hints
    ) -> Iterable[dict]:
        return vertices[type_name]

    def project_property(
        self,
        lineages_and_other_data: Iterable[Tuple[LineageToken, Any]], 
        field_name: str,
        **hints
    ) -> Iterable[Tuple[LineageToken, Any]]:
        return (
            (lineage, (lineage.token[field_name], other_data))
            for lineage, other_data in lineages_and_other_data
        )

    def project_neighbors(
        self,
        lineages_and_other_data: Iterable[Tuple[LineageToken, Any]], 
        direction: str,
        edge_name: str, 
        **hints
    ) -> Iterable[Tuple[LineageToken, Iterable[DataToken], Any]]:
        edge_info = edges[edge_name]
        neighbor_tokens = None
        
        for lineage, other_data in lineages_and_other_data:
            uuid = lineage.token['uuid']
            if direction == 'out':
                neighbor_tokens = [
                    vertices_by_uuid[destination_uuid]
                    for source_uuid, destination_uuid in edge_info
                    if source_uuid == uuid
                ]
            elif direction == 'in':
                neighbor_tokens = [
                    vertices_by_uuid[destination_uuid]
                    for source_uuid, destination_uuid in edge_info
                    if destination_uuid == uuid
                ]
            else:
                raise AssertionError()
                
            yield (lineage, neighbor_tokens, other_data)

    def coerce_to_type(
        self,
        lineages_and_other_data: Iterable[Tuple[LineageToken, Any]], 
        type_name: str,
        **hints
    ) -> Iterable[Tuple[LineageToken, Any]]:
        # no-op
        return lineages_and_other_data

In [11]:
schema = build_ast_schema(parse(SCHEMA_TEXT))

In [12]:
query = '''
{
    Animal {
        name @output(out_name: "animal_name")
        uuid @output(out_name: "animal_uuid")
    }
}
'''
query_arguments = {}

In [13]:
query = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @output(out_name: "child_name")
        }
    }
}
'''
query_arguments = {}

In [28]:
query = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}
'''
query_arguments = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [30]:
query = '''
{
    Animal {
        name @output(out_name: "grandparent_name")

        out_Animal_ParentOf {
            name @output(out_name: "parent_name")
            
            out_Animal_ParentOf {
                name @output(out_name: "child_name")
            }
        }
    }
}
'''
query_arguments = {}

In [31]:
ir_and_metadata = graphql_to_ir(schema, query)
result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
result

[{'grandparent_name': 'Pongo',
  'parent_name': 'Dipstick',
  'child_name': 'Domino'},
 {'grandparent_name': 'Pongo',
  'parent_name': 'Dipstick',
  'child_name': 'Little Dipper'},
 {'grandparent_name': 'Pongo',
  'parent_name': 'Dipstick',
  'child_name': 'Oddball'},
 {'grandparent_name': 'Perdy',
  'parent_name': 'Dipstick',
  'child_name': 'Domino'},
 {'grandparent_name': 'Perdy',
  'parent_name': 'Dipstick',
  'child_name': 'Little Dipper'},
 {'grandparent_name': 'Perdy',
  'parent_name': 'Dipstick',
  'child_name': 'Oddball'}]